# Day 2 - Lab 2: Documenting Key Decisions with ADRs (Solution)

**Objective:** Use an LLM as a research assistant to compare technical options and synthesize the findings into a formal, version-controlled Architectural Decision Record (ADR).

**Introduction:**
This solution notebook provides the complete prompts and explanations for the ADR generation lab. It demonstrates how to use an LLM for comparative research and then synthesize that research into a structured, formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, recommended_models_table

client, model_name, api_provider = setup_llm_client(model_name="gpt-5-2025-08-07")

✅ LLM Client configured: Using 'openai' with model 'gpt-5-2025-08-07'


In [2]:
recommended_models_table()

| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|
| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| codex-mini-latest | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |
| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |
| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 0 | 0 |
| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-live-001 | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.5-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-image-preview | google | ✅ | ✅ | ❌ | 32,768 | 32,768 |
| gemini-2.5-flash-lite | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-live-2.5-flash-preview | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-veo-3 | google | ✅ | ❌ | ❌ | - | - |
| google-cloud/speech-to-text/latest_long | google | ❌ | ❌ | ✅ | - | - |
| google-cloud/speech-to-text/latest_short | google | ❌ | ❌ | ✅ | - | - |
| gpt-4.1 | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-mini | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.5 | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-image-1 | openai | ✅ | ✅ | ❌ | - | - |
| imagen-3.0-generate-002 | google | ❌ | ✅ | ❌ | - | - |
| imagen-4.0-generate-001 | google | ❌ | ✅ | ❌ | 480 | - |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |
| tokyotech-llm/Llama-3.1-Swallow-70B-Instruct-v0.3 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| whisper-1 | openai | ❌ | ❌ | ✅ | - | - |

'| Model | Provider | Vision | Image Gen | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|\n| claude-opus-4-1-20250805 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ❌ | ❌ | 1,000,000 | 100,000 |\n| codex-mini-latest | openai | ✅ | ❌ | ❌ | 200,000 | 100,000 |\n| dall-e-3 | openai | ❌ | ✅ | ❌ | - | - |\n| deepseek-ai/DeepSeek-V3 | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-V3-Small | huggingface | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| deepseek-ai/DeepSeek-VL2 | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Small | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/DeepSeek-VL2-Tiny | huggingface | ✅ | ❌ | ❌ | 32,000 | 8,000 |\n| deepseek-ai/Janus-Pro-7B | huggingface | ✅ | ❌ | ❌ | 0 | 0 |\n| gemini-2.0-flash | google | ✅ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.0-flash-lite 

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): The ADR Template

**Explanation:**
This prompt asks the LLM to generate a standard markdown template for an ADR. The key is to be specific about the sections required (`Title`, `Status`, `Context`, `Decision`, `Consequences`), which guides the LLM to produce a well-structured and reusable template.

In [3]:
adr_template_prompt = """You are a principal engineer who champions clear documentation. Generate a concise, reusable markdown template for an Architectural Decision Record (ADR).

The template must include the following sections:
- # Title: [A short, descriptive title for the decision]
- **Status:** [Proposed | Accepted | Deprecated | Superseded]
- ## Context
  - [Describe the problem, the driving forces, and the constraints.]
- ## Decision
  - [State the chosen solution clearly and concisely.]
- ## Consequences
  - [List the positive outcomes, negative trade-offs, and any future work required.]"""

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md")

--- Generating ADR Template ---
```markdown
# Title: [A short, descriptive title for the decision]

**Status:** [Proposed | Accepted | Deprecated | Superseded]

## Context
- Problem: [Describe the problem.]
- Driving forces: [Key goals, quality attributes, stakeholders, deadlines.]
- Constraints: [Technical, organizational, legal, budget, legacy, etc.]

## Decision
- [State the chosen solution clearly and concisely.]
- Rationale: [Why this option over others; key trade-offs considered.]

## Consequences
- Positive outcomes: [Benefits and desired results.]
- Negative trade-offs: [Costs, risks, compromises, new limitations.]
- Future work: [Follow-ups, migration/cleanup tasks, monitoring, review date.]
```
✅ Successfully saved artifact to: templates/adr_template.md


### Challenge 2 (Intermediate): AI-Assisted Research

**Explanation:**
This prompt leverages the LLM's vast training data to perform a comparative analysis. By instructing it to be an "unbiased research assistant" and asking for "pros and cons for each," we guide the model to provide a balanced view rather than a simple recommendation. This produces a more valuable and objective input for our own decision-making process.

In [4]:
db_research_prompt = """You are an unbiased research assistant. Your task is to provide a balanced technical comparison for a software development team.

For the use case of a new hire onboarding tool that needs a semantic search feature, compare and contrast the following two approaches:

1.  **Approach 1:** Using PostgreSQL with the `pgvector` extension.
2.  **Approach 2:** Using a specialized, dedicated vector database (e.g., ChromaDB, FAISS, Weaviate).

Please provide a summary of the pros and cons for each approach. Consider factors like operational complexity, cost, query flexibility, and scalability for a small-to-medium sized enterprise application.
"""

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

--- Researching Database Options ---
Below is a balanced comparison for a small-to-medium enterprise onboarding tool that needs semantic search.

Approach 1: PostgreSQL with pgvector

Pros
- Low operational overhead and cost
  - One datastore to run, back up, monitor, and secure (you likely already run Postgres).
  - No extra service or network hop; easier local/dev parity.
  - Managed Postgres offerings (AWS RDS, GCP Cloud SQL, Azure) commonly support pgvector.
- Strong query flexibility
  - Full SQL with joins, filters, aggregations, window functions, and transactions.
  - Easy to combine semantic search with relational filters (department, role, region) and permissions (row-level security).
  - Hybrid search is straightforward by combining pgvector with Postgres full-text (tsvector/tsrank) or trigram.
- Data integrity and governance
  - ACID, migrations, backups (PITR), auditing, RLS; mature tooling and compliance posture.
  - Store document, metadata, and embedding in one place; up

### Challenge 3 (Advanced): Synthesizing the ADR

**Explanation:**
This prompt demonstrates a powerful synthesis task. We provide the LLM with two key inputs: unstructured information (the research) and a desired structure (the template). The agent's job is to merge them, creating a polished, formal document. This is a repeatable pattern for turning raw analysis into professional documentation. By assigning the persona of a Staff Engineer, we guide the LLM to adopt a formal and authoritative tone suitable for an official project artifact.

In [5]:
adr_template = load_artifact("templates/adr_template.md")

synthesis_prompt = f"""You are a Staff Engineer responsible for documenting key architectural decisions.

Your task is to populate the provided ADR template to formally document the decision to **use PostgreSQL with the pgvector extension** for our project.

Use the research provided below to fill in the 'Context' and 'Consequences' sections of the template. Be thorough and objective, summarizing the key points from the research.

--- ADR TEMPLATE ---
{adr_template}
--- END TEMPLATE ---

--- RESEARCH CONTEXT ---
{db_research_output}
--- END CONTEXT ---

The final ADR should be complete and ready for review.
"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md")
else:
    print("Skipping ADR synthesis because template or research is missing.")

--- Synthesizing Final ADR ---
# Title: Adopt PostgreSQL with pgvector for semantic search and vector storage

Status: Accepted

## Context
- Problem:
  - The onboarding tool needs semantic search over documents/chunks while enforcing relational filters (e.g., department, role, region) and access controls.
  - We must keep operational overhead low and maintain transactional consistency between source content, metadata, and embeddings.

- Driving forces:
  - Minimize operational complexity and cost (ideally a single datastore to run, back up, secure, and monitor).
  - Strong query flexibility: combine vector similarity with SQL joins, filters, aggregations, and transactions.
  - Governance and data integrity: ACID guarantees, migrations, backups (including PITR), auditing, and row-level security for multi-tenant or role-based access.
  - Hybrid search capability (dense vectors + full-text/trigram) within one system.
  - Adequate performance for a small-to-medium enterprise workload (up 

## Lab Conclusion

Well done! You have used an LLM to automate a complex but critical part of the architectural process. You leveraged its vast knowledge base for research and then used it again for synthesis, turning raw analysis into a formal, structured document. This `adr_001_database_choice.md` file now serves as a permanent, valuable record for anyone who works on this project in the future.

> **Key Takeaway:** The pattern of **Research -> Synthesize -> Format** is a powerful workflow. You can use an LLM to gather unstructured information and then use it again to pour that information into a structured template, creating high-quality, consistent documentation with minimal effort.